In [17]:
%env CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
%env USER=neo4j
%env PW=TeamWang

env: CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
env: USER=neo4j
env: PW=TeamWang


In [18]:
CONNECT="$CYPHER -u $USER -p $PW"


In [19]:
!{CONNECT} "SHOW DATABASES"

+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

2 rows available after 75 ms, consumed after another 29 ms


In [20]:
!$CONNECT "MATCH (n) DETACH DELETE n"

0 rows available after 1086 ms, consumed after another 0 ms
Deleted 129444 nodes, Deleted 220183 relationships


load the airbnb data into Neo4j

In [21]:
!cat /home/jupyter/airbnb/load_data.cypher | {CONNECT} --format plain

COUNT(l)
5835
COUNT(a)
42
COUNT(n)
41
COUNT(h)
4633
COUNT(u)
55917
COUNT(r)
62976


returning a total node count

In [22]:
!{CONNECT} "MATCH (n) RETURN count(n)"


+----------+
| count(n) |
+----------+
| 129444   |
+----------+

1 row available after 2 ms, consumed after another 0 ms


run a count for each unique node label in the graph

In [23]:
!{CONNECT} "MATCH (n) RETURN distinct labels(n), count(n)"

+-----------------------------+
| labels(n)        | count(n) |
+-----------------------------+
| ["Listing"]      | 5835     |
| ["Amenity"]      | 42       |
| ["Neighborhood"] | 41       |
| ["Host"]         | 4633     |
| ["User"]         | 55917    |
| ["Review"]       | 62976    |
+-----------------------------+

6 rows available after 1 ms, consumed after another 64 ms


sample the data by returning the contents of 10 random nodes in one query

In [24]:
!{CONNECT} "MATCH (n) RETURN n LIMIT 10"

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| n                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Listing {bedrooms: 1, listing_id: "72635", availability_365: 240, price: 300.0, accommodates: 6, name: "3 Private Bedrooms, SW Austin", property_type: "House", bathrooms: 2})                                      |
| (:Listing {bedrooms: 1, cleaning_fee: 75.0, weekly_price: 600.0, listing_id: "5386323", availability_365: 364, price: 99.0, ac

sample the data by returning the contents of 10 random relationships in one query

In [25]:
!{CONNECT} "MATCH ()-[r]->() RETURN r LIMIT 10"

+------------+
| r          |
+------------+
| [:REVIEWS] |
| [:HOSTS]   |
| [:HOSTS]   |
| [:HOSTS]   |
| [:HOSTS]   |
| [:HOSTS]   |
| [:HOSTS]   |
| [:HOSTS]   |
| [:REVIEWS] |
| [:REVIEWS] |
+------------+

10 rows available after 1 ms, consumed after another 2 ms


Q1. How many hosts are located in "Austin, Texas, United States"?

In [26]:
!{CONNECT} "MATCH (h:Host {{location: 'Austin, Texas, United States'}}) RETURN count(h)"

+----------+
| count(h) |
+----------+
| 3774     |
+----------+

1 row available after 3 ms, consumed after another 6 ms


Q2. Which listings does host_id = "4641823" have? Return the listing name, property_type,
price, and availability_365 sorted by price. Limit the results to 10.

In [27]:
!{CONNECT} "MATCH (:Host {{host_id: '4641823'}})-[]-(l) RETURN l.name, l.property_type, l.price, l.availability_365 ORDER BY l.price LIMIT 10"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "1BR Convenient Austin Condo "        | "Apartment"     | 93.0    | 354                |
| "1BR Inviting Downtown Condo, Pool"   | "Apartment"     | 99.0    | 335                |
| "2BR/1.5BA Charming House Off SoCo"   | "House"         | 110.0   | 357                |
| "2BR Prime East-Side Downtown"        | "House"         | 121.0   | 341                |
| "1BR SoCo Treehouse Studio"           | "House"         | 129.0   | 327                |
| "1BR/1.5BA East 6th, Colorful 2Story" | "Apartment"     | 134.0   | 344                |
| "3BR Prestigious Home Travis Heights" | "House"         | 138.0   | 0                  |
| "1BR/1.5BA Perfectly Located Casita"  | "House"         | 140.0   | 351                |

Q3. Which users wrote a review for listing_id = "5293632"? Return the user’s id and name
sorted alphabetically by name. Limit the results to 10.


In [28]:
!{CONNECT} "MATCH (u)-[:WROTE]->(:Review)-[:REVIEWS]->(:Listing {{listing_id: '5293632'}}) RETURN u.user_id, u.name ORDER BY u.name LIMIT 10"

+--------------------------------+
| u.user_id  | u.name            |
+--------------------------------+
| "18286390" | "Annie"           |
| "30193020" | "Carole"          |
| "16497582" | "Cory"            |
| "35022795" | "Craig And Trina" |
| "13281665" | "Dianne"          |
| "29601600" | "Hannah"          |
| "11940539" | "Jacob"           |
| "3213433"  | "Jessie"          |
| "41722221" | "Johannes"        |
| "28480571" | "Ju-Ju"           |
+--------------------------------+

10 rows available after 2 ms, consumed after another 1 ms


Q4. Which users wrote a review for any listing which has the amenities "Washer" and "Dryer"?
Return the user’s id and name sorted alphabetically by name. Limit the results to 10.¶

In [29]:
!{CONNECT} "MATCH (:Amenity {{name:'Washer'}})<-[:HAS]-(l:Listing)-[:HAS]->(:Amenity {{name:'Dryer'}}) with l MATCH (u)-[:WROTE]->(:Review)-[:REVIEWS]->(l) RETURN u.user_id, u.name ORDER BY u.name LIMIT 10"

+-------------------------------------+
| u.user_id  | u.name                 |
+-------------------------------------+
| "6524431"  | "'Ley"                 |
| "8026901"  | "(We Are) Bonnie & Ky" |
| "7802424"  | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
| "5164376"  | "(email hidden)"       |
| "13381969" | "(email hidden)"       |
| "7802424"  | "(email hidden)"       |
| "11656027" | "(email hidden)"       |
| "15315643" | "(email hidden)"       |
+-------------------------------------+

10 rows available after 2 ms, consumed after another 199 ms


Q5. Which listings have 3 bedrooms and are located in the Clarksville neighborhood? Return
the listing name, property_type, price, and availability_365 sorted by price. Limit the results to 5.

In [30]:
!{CONNECT} "MATCH (l:Listing)-[:IN_NEIGHBORHOOD]->(n:Neighborhood {{name:'Clarksville'}}) where l.bedrooms = 3 RETURN l.name, l.property_type, l.price, l.availability_365 ORDER BY l.price LIMIT 5"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "3BR/2.5BA Exquisite Townhouse"       | "House"         | 222.0   | 358                |
| "3BR/2.5BA Tarrytown Duplex, Austin!" | "House"         | 249.0   | 336                |
| "Austin downtown hideaway"            | "House"         | 249.0   | 364                |
| "3BD Luxury Cottage by Lake Austin"   | "House"         | 290.0   | 309                |
| "Entire Adorable Downtown House"      | "House"         | 295.0   | 309                |
+----------------------------------------------------------------------------------------+

5 rows available after 2 ms, consumed after another 3 ms


Q6. Which amenities are the most common? Return the name of the amenity and its frequency.
 Sort the results by count in descending order. Limit the results to 5.

In [31]:
!{CONNECT} "MATCH (:Listing)-[]->(a:Amenity) RETURN a.name, count(a) ORDER BY count(a) DESC LIMIT 5"

+---------------------------------------+
| a.name                     | count(a) |
+---------------------------------------+
| "Air Conditioning"         | 5615     |
| "Wireless Internet"        | 5479     |
| "Heating"                  | 5440     |
| "Kitchen"                  | 5400     |
| "Free Parking on Premises" | 5123     |
+---------------------------------------+

5 rows available after 2 ms, consumed after another 230 ms


Q7. Which neighborhoods have the highest number of listings? Return the neighborhood’s
name and zip code (neighborhood_id) along with the number of listings they have sorted
by the number of listings in descending order. Limit the results to 5.

In [32]:
!{CONNECT} "MATCH (l:Listing)-[]->(n:Neighborhood) RETURN n.name, n.neighborhood_id, count(l) ORDER BY count(l) DESC LIMIT 5"

+-------------------------------------------------+
| n.name           | n.neighborhood_id | count(l) |
+-------------------------------------------------+
| NULL             | "78704"           | 1601     |
| NULL             | "78702"           | 797      |
| "Clarksville"    | "78703"           | 419      |
| "East Riverside" | "78741"           | 414      |
| NULL             | "78745"           | 328      |
+-------------------------------------------------+

5 rows available after 2 ms, consumed after another 25 ms
